# Desafio Kinea


## Importando e definindo constantes globais

In [32]:
import requests
import base64 as b64
import json
from typing import Dict, List
import csv
from dateutil.relativedelta import relativedelta

In [4]:
import logging
logging.level=logging.INFO

In [5]:
from datetime import date, datetime
from workalendar.america import BrazilBankCalendar

In [6]:
class BrazilAmbima(BrazilBankCalendar):
    include_ash_wednesday = False

    def get_variable_days(self, year):
      """
      Define the brazilian variable holidays and the last
      day for only internal bank transactions
      """
      days = super().get_variable_days(year)[:-1]
      return days
cal = BrazilAmbima()

In [7]:
client_id = "9r2j5fK3DntV"
client_secret = "ufaORoKwRRmw"

## Funções básicas

In [8]:
def calculate_du(start_date, end_date) -> int:
  """O formato de data é date(AAAA, MM, DD), como date(2022, 2, 4) para 4 de fev de 2022"""
  interval = cal.get_working_days_delta(start_date, end_date)+1
  return interval

In [9]:
def b64encode(data:str) -> str:
  "Codifica uma string em base64"
  encodedBytes = b64.b64encode(data.encode("utf-8"))
  encodedStr = str(encodedBytes, "utf-8")
  return encodedStr

In [10]:
def check_error(acquired_value, expected_value):
  err = abs((acquired_value-expected_value)/expected_value)*100
  err_str = f'{err:0.3}%'
  return err_str

## Interface com API Ambima

In [11]:
def get_access_token(_id:str, secret:str) -> requests.models.Response:
  """Recebe id e secret de um dado app e retorna o toke de acesso para aquele app"""
  address = "https://api.anbima.com.br/oauth/access-token?"
  credentials = b64encode(f"{_id}:{secret}")
  body = {"grant_type": "client_credentials"}
  headers = {
    "Authorization": f"Basic {credentials}",
    "Content-Type": "application/json"
  }
  logging.debug(f"Auth headers: {headers}")
  res = requests.post(address, headers=headers, json=body)
  logging.info(f"Auth res status: {res.status_code}")
  logging.debug(f"Auth res text: {res.text}")
  return res


In [12]:
def get_data(_id:str, token:str, address:str) -> requests.models.Response:
  headers = {
    "Content-Type": "application/json",
    "access_token": token,
    "client_id": _id}
  logging.debug(f"Auth headers: {headers}")
  res = requests.get(address, headers=headers)
  logging.info(f"Auth res status: {res.status_code}")
  if res.status_code == 401:
    new_token = get_access_token(_id, client_secret)
    get_data(_id, new_token, address)
  logging.debug(f"Auth res text: {res.text}")
  return res

In [13]:
auth_res = get_access_token(client_id, client_secret)
access_token = json.loads(auth_res.content)["access_token"]
logging.info(f"Access Token: {access_token}")

In [14]:
address_base = "https://api-sandbox.anbima.com.br/feed/precos-indices/v1"
address_mercado_secundario_TPF = address_base+"/titulos-publicos/mercado-secundario-TPF"

data_res = get_data(client_id, access_token, address_mercado_secundario_TPF)
data_list = json.loads(data_res.content)
logging.info(data_list)

In [15]:
def save_to_csv(filedata:List[Dict], filename:str):
  """Take a list of dicts and save as an .csv with the given name"""
  keys = filedata[0].keys()
  with open('exports\\'+filename, 'w', newline='') as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writeheader()
      dict_writer.writerows(filedata)

In [16]:
def clean_mercado_secundario(list_to_clean:List[Dict]):
  keep_keys = ['tipo_titulo','data_referencia','data_vencimento','data_base','taxa_indicativa','pu']
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [17]:
clean_mercado = clean_mercado_secundario(data_list)

## Cálculo de PUs

In [18]:
def get_dias_cupom(start_date, end_date):
  pgto_date_list = []
  pgto_date = end_date

  # Gera da lista de datas a partir da data final, em incrementos de 6 meses
  while pgto_date > start_date:
    pgto_date_list.append(pgto_date)
    pgto_date -= relativedelta(months=6)

  # Inverte a ordem da lista para crescente
  pgto_date_list=pgto_date_list[::-1]
  
  return pgto_date_list

LTN

In [19]:
def calcular_ltn(taxa_anual, data_liqui, data_venc, valor_face=1000, expected_pu=0, debug=False):
  du = calculate_du(data_liqui, data_venc)
  pu = valor_face/((1+taxa_anual)**(du/252))
  if debug:
    return(f'PU: {pu} | Erro: {check_error(pu, expected_pu)}')
  return pu

In [20]:
# calcular_ltn(0.1212, date(2022,3,21), date(2025,1,1))
calcular_ltn(0.0728, date(2017,10,20), date(2018,1,1))

986.7040298582051

NTN-F

In [21]:
def calcular_ntn_f(taxa_anual, taxa_cupom, data_liqui, data_venc, valor_face=1000):
  cupom_total = 0
  upper_prazo_medio = 0
  upper_duration = 0
  lower_prazo_medio = 0
  lower_duration = 0
  
  for cupom in get_dias_cupom(data_liqui, data_venc):
    du_cupom = calculate_du(data_liqui, cupom)
    pgto_cupom_pm = valor_face*((1+taxa_cupom)**(1/2)-1)
    pgto_cupom_duration = valor_face*((1+taxa_cupom)**(1/2)-1)/((1+taxa_anual)**(du_cupom/252))
    cupom_total += pgto_cupom_duration
    dias_remanescentes = (data_venc-cupom).days

    upper_prazo_medio += dias_remanescentes*pgto_cupom_pm
    lower_prazo_medio += pgto_cupom_pm
    upper_duration += dias_remanescentes*pgto_cupom_duration
    lower_duration += pgto_cupom_duration

  du = calculate_du(data_liqui, data_venc)
  pagamento_final = valor_face/((1+taxa_anual)**(du/252))

  pu = pagamento_final + cupom_total
  prazo_medio = upper_prazo_medio/(lower_prazo_medio+valor_face)
  duration = upper_duration/(lower_duration+pagamento_final)
  
  return pu, prazo_medio, duration

In [33]:
pu, prazo_medio, duration = calcular_ntn_f(0.122, 0.1, date(2022,3,21), date(2033,1,1)) # PM 2888, D 1623

# Prazo médio é fluxo futuro apenas, então não se usa taxa_anual para cálculo de prazo médio 
# Duration é sempre menor que prazo médio
print(f'pu: {pu} | prazo médio: {prazo_medio} | duration: {duration}')
print(f'Erro pu: {check_error(pu, 897.21734)}')
print(f'Erro prazo médio: {check_error(prazo_medio, 2888)}')
print(f'Erro duration: {check_error(duration, 1623)}')

pu: 897.1143129880454 | prazo médio: 993.4549072071991 | duration: 1577.2703260560822
Erro pu: 0.0115%
Erro prazo médio: 65.6%
Erro duration: 2.82%


NTN-B Principal

In [23]:
def calcular_ntn_b_princ(taxa_anual, data_liqui, data_venc, infla_proj, valor_face=1000):
  d15_base = date(data_liqui.year, data_liqui.month, 15)
  d15_proj = d15_base + relativedelta(months=1)
  
  fator = 3.849352 # descobrir como calcular
  vna = valor_face*fator
  vna_proj = vna*(1+infla_proj)**((data_liqui-d15_base)/(d15_proj-d15_base))
  
  # cupom_total = 0
  # upper_prazo_medio = 0
  # lower_prazo_medio = 0
  # upper_duration = 0
  # lower_duration = 0

  # for cupom in get_dias_cupom(data_liqui, data_venc):
  #   du_cupom = calculate_du(data_liqui, cupom)
  #   pgto_cupom_pm = vna_proj*((1+taxa_cupom)**(1/2)-1)
  #   pgto_cupom_duration = vna_proj*((1+taxa_cupom)**(1/2)-1)/((1+taxa_anual)**(du_cupom/252))
  #   cupom_total += pgto_cupom_duration

  #   dias_remanescentes = (cupom-data_liqui).days

  #   upper_prazo_medio += dias_remanescentes*pgto_cupom_pm
  #   lower_prazo_medio += pgto_cupom_pm
  #   upper_duration += dias_remanescentes*pgto_cupom_duration
  #   lower_duration += pgto_cupom_duration
  
  du = calculate_du(data_liqui, data_venc)
  pu = vna_proj/((1+taxa_anual)**(du/252))
  
  # prazo_medio = upper_prazo_medio/(lower_prazo_medio+valor_face)
  # duration = upper_duration/(lower_duration+cupom_total)
  
  return pu

In [24]:
ntn_b_princ_pu = calcular_ntn_b_princ(0.0576, date(2022,3,21), date(2045,5,15), 0.0107)

# print(f'ntn_b_princ_pu: {ntn_b_princ_pu} | ntn_b_princ_prazo_medio: {ntn_b_princ_prazo_medio} | ntn_b_princ_duration: {ntn_b_princ_duration}')
print(f'ntn_b_princ_pu: {ntn_b_princ_pu}')
print(f'Erro ntn_b_princ_pu: {check_error(ntn_b_princ_pu, 1059.153)}')

ntn_b_princ_pu: 1058.917931176815
Erro ntn_b_princ_pu: 0.0222%


NTN-B

In [25]:
def calcular_ntn_b(taxa_anual, taxa_cupom, data_liqui, data_venc, infla_proj, valor_face=1000):
  d15_base = date(data_liqui.year, data_liqui.month, 15)
  d15_proj = d15_base + relativedelta(months=1)
  
  fator = 3.849352 # descobrir como calcular
  vna = valor_face*fator
  vna_proj = vna*(1+infla_proj)**((data_liqui-d15_base)/(d15_proj-d15_base))
  
  cupom_total = 0
  upper_prazo_medio = 0
  lower_prazo_medio = 0
  upper_duration = 0
  lower_duration = 0

  for cupom in get_dias_cupom(data_liqui, data_venc):
    du_cupom = calculate_du(data_liqui, cupom)
    pgto_cupom_pm = vna_proj*((1+taxa_cupom)**(1/2)-1)
    pgto_cupom_duration = vna_proj*((1+taxa_cupom)**(1/2)-1)/((1+taxa_anual)**(du_cupom/252))
    cupom_total += pgto_cupom_duration

    dias_remanescentes = (cupom-data_liqui).days

    upper_prazo_medio += dias_remanescentes*pgto_cupom_pm
    lower_prazo_medio += pgto_cupom_pm
    upper_duration += dias_remanescentes*pgto_cupom_duration
    lower_duration += pgto_cupom_duration

  du = calculate_du(data_liqui, data_venc)
  pu = cupom_total + vna_proj/((1+taxa_anual)**(du/252))
  
  prazo_medio = upper_prazo_medio/(lower_prazo_medio+valor_face)
  duration = upper_duration/(lower_duration+cupom_total)
  
  return pu, prazo_medio, duration

In [26]:
ntn_b_pu, ntn_b_prazo_medio, ntn_b_duration = calcular_ntn_b(0.0556, 0.06, date(2022,3,18), date(2032,8,15), 0.0094)

print(f'ntn_b_pu: {ntn_b_pu} | ntn_b_prazo_medio: {ntn_b_prazo_medio} | ntn_b_duration: {ntn_b_duration}')
print(f'Erro ntn_b_pu: {check_error(ntn_b_pu, 4006.774)}')
# print(f'Erro ntn_b_prazo_medio: {check_error(ntn_b_prazo_medio, 2888)}')
# print(f'Erro ntn_b_duration: {check_error(ntn_b_duration, 1623)}')

ntn_b_pu: 4006.5074402667087 | ntn_b_prazo_medio: 1394.1463583332861 | ntn_b_duration: 898.7090699567603
Erro ntn_b_pu: 0.00665%


LTF

In [27]:
def calcular_ltf(taxa_anual, data_liqui, data_venc, vna, valor_face=1000):
  ltn = calcular_ltn(taxa_anual, data_liqui, data_venc, vna)
  return ltn

In [28]:
calcular_ltf(0.001193, date(2022,5,12), date(2022,9,1), 11648.93567)

11644.527330634757

## Cálculo Geral

In [ ]:
for titulo in clean_mercado:
  data_liqui=datetime.strptime(titulo['data_referencia'], "%Y-%m-%d").date()
  data_venc=datetime.strptime(titulo['data_vencimento'], "%Y-%m-%d").date()

  if titulo['tipo_titulo'] == 'LTN':
    pu_calc = calcular_ltn(
      taxa_anual=titulo['taxa_indicativa']/100,
      data_liqui=data_liqui,
      data_venc=data_venc)
    titulo['pu_calc']=pu_calc

  elif titulo['tipo_titulo'] == 'NTN-B':
    pu_calc, pm, dur = calcular_ntn_b(
      data_liqui=data_liqui,
      data_venc=data_venc,
      taxa_anual=titulo['taxa_indicativa']/100,
      # taxa_cupom=titulo['taxa_indicativa']/100, Esse deve ser um valor fixo para o NTN-B. Toda taxa cupom é fixa para um tipo de titulo
      infla_proj=0.0094)
    titulo['pu_calc']=pu_calc
    titulo['prazo_medio']=pm
    titulo['duration']=dur

print([x for x in clean_mercado if x['tipo_titulo'] in ['LTN','NTN-B']])